# 1. Подготовка

Хотел бы не тратить время на лемматизацию при проработке вариантов

In [1]:
import pandas as pd
import numpy as np
"""data = pd.read_csv('/datasets/toxic_comments.csv')
data"""

"data = pd.read_csv('/datasets/toxic_comments.csv')\ndata"

In [2]:
from pymystem3 import Mystem
m = Mystem()
"""for i in range(data.shape[0]):
    if i%10000==0:
        print(i)
    data.loc[i, 'lemmatized'] = ''.join(m.lemmatize((''.join(re.sub(r'[^a-zA-Z]', ' ', data.loc[i, 'text'].lower())))))[:-1]
data"""
pass

In [1]:
"""data.to_csv('toxic_data.csv')"""
pass

In [2]:
data = pd.read_csv('toxic_data.csv', index_col=0)
data

,text,toxic,lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m s...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on im...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,and for the second time of asking when ...
159567,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that is a ...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm theres no actual article for pr...
159569,And it looks like it was actually you who put ...,0,and it looks like it was actually you who put ...


Предлагаю разделить датасет на трейн и тест заранее

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size=0.8)
train

,text,toxic,lemmatized
28291,hey guess what!!!! BS again buddy. i know how ...,0,hey guess what bs again buddy i know how ...
91597,Austin Bourke was a meteorologist. He was invo...,0,austin bourke was a meteorologist he was invo...
67544,Don't hide the incriminating photographs you N...,1,don t hide the incriminating photographs you n...
131756,"Friendly Discussion ==\n\nDear Kumarrao,\n\nI ...",0,friendly discussion dear kumarrao i had ...
136401,"So, for this limited goal (dimension is time o...",0,so for this limited goal dimension is time o...
...,...,...,...
108822,I'm all for changing it to BCE and CE from BC ...,0,i m all for changing it to bce and ce from bc ...
83278,"""\n\nI was blocked for harrassment and disrupt...",0,i was blocked for harrassment and disruptio...
136848,"""\n\n Example \n\nOkay, heres what you should ...",0,example okay heres what you should do ...
74109,"Invitation \n\nG'day Thewanderer, just wanted ...",0,invitation g day thewanderer just wanted to...


In [4]:
train_toxic = train[train['toxic']==1]
train_toxic

,text,toxic,lemmatized
67544,Don't hide the incriminating photographs you N...,1,don t hide the incriminating photographs you n...
127285,I dont like you\nYou are like a Wiki Nazi alwa...,1,i dont like you you are like a wiki nazi alway...
24263,Wikipedia is the worst idea ever. It will nev...,1,wikipedia is the worst idea ever it will nev...
145464,Your DO know that Hispanics are part of Europe...,1,your do know that hispanics are part of europe...
109421,Your country is shit!!! All it does is pollute...,1,your country is shit all it does is pollute...
...,...,...,...
65988,Why Wikpedia Is Gay\n\nWikipedia sucks. I will...,1,why wikpedia is gay wikipedia sucks i will n...
53272,YO\nWHY CANT I MESS UP UR SITE\nNO SCHOOLS R A...,1,yo why cant i mess up ur site no schools r all...
121419,666 = 2x above Holy 333 Trinity \n\nrepent ba...,1,x above holy trinity repent bastard
79281,"""\n\n pakis - one of the most ugliest individu...",1,pakis one of the most ugliest individual...


In [5]:
words_used_in_toxic = []
words_used_in_nontoxic = []

for i in train.index:
    buf = train.loc[i]
    phrase = set(buf['lemmatized'].split())
    if buf['toxic']==1:
        words_used_in_toxic+=list(phrase)
    else:
        words_used_in_nontoxic+=list(phrase)
toxic_dict = pd.Series(words_used_in_toxic).value_counts()
nontoxic_dict = pd.Series(words_used_in_nontoxic).value_counts()
print(len(words_used_in_toxic), len(words_used_in_nontoxic))

406735 5281081


In [6]:
words_used_in_toxic = set(words_used_in_toxic)
words_used_in_nontoxic = set(words_used_in_nontoxic)

In [7]:
words_used_ONLY_in_toxic = words_used_in_toxic - words_used_in_nontoxic
words_used_ONLY_in_nontoxic = words_used_in_nontoxic - words_used_in_toxic
words_used_everywhere = words_used_in_toxic & words_used_in_nontoxic

In [8]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
words_used_everywhere-=set(stop_words)

In [9]:
toxic_dict = dict(toxic_dict)
nontoxic_dict = dict(nontoxic_dict)

In [10]:
quantity = pd.Series()
for word in list(words_used_everywhere):
    quantity[word] = toxic_dict[word]/nontoxic_dict[word]
quantity

country       0.084746
treatment     0.035714
apologises    0.333333
spartacus     0.166667
musicians     0.020408
                ...   
activists     0.046154
userpages     0.057692
wba           0.500000
provided      0.033230
cult          0.120000
Length: 22452, dtype: float64

In [11]:
train_copy = train.copy()
train_copy.loc[:, 'lemm_set'] = 0
j=0
for i in train_copy.index:
    if j%1000==0:
        print(j)
    j+=1
    train_copy.loc[i, 'lemm_set'] = '#'.join(list(set(train_copy.loc[i, 'lemmatized'].split())))
train_copy

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000


,text,toxic,lemmatized,lemm_set
28291,hey guess what!!!! BS again buddy. i know how ...,0,hey guess what bs again buddy i know how ...,noone#a#say#spend#can#hey#they#to#its#mason#kn...
91597,Austin Bourke was a meteorologist. He was invo...,0,austin bourke was a meteorologist he was invo...,weird#visitation#agricultural#a#great#thesis#a...
67544,Don't hide the incriminating photographs you N...,1,don t hide the incriminating photographs you n...,trying#incriminating#think#to#graphic#its#phot...
131756,"Friendly Discussion ==\n\nDear Kumarrao,\n\nI ...",0,friendly discussion dear kumarrao i had ...,historian#always#of#distinguish#can#p#urs#much...
136401,"So, for this limited goal (dimension is time o...",0,so for this limited goal dimension is time o...,format#may#reduced#sortkey#recognised#could#ca...
...,...,...,...,...
108822,I'm all for changing it to BCE and CE from BC ...,0,i m all for changing it to bce and ce from bc ...,bc#i#changing#m#all#it#from#bce#for#and#to#ce#ad
83278,"""\n\nI was blocked for harrassment and disrupt...",0,i was blocked for harrassment and disruptio...,removes#indicating#wished#block#agreed#quite#o...
136848,"""\n\n Example \n\nOkay, heres what you should ...",0,example okay heres what you should do ...,put#ask#example#do#monkey#ref#can#could#okay#e...
74109,"Invitation \n\nG'day Thewanderer, just wanted ...",0,invitation g day thewanderer just wanted to...,send#us#merrier#to#join#invite#at#g#over#day#j...


In [93]:
from sklearn.metrics import f1_score

for itr in [8]: 
    print('*'*25, itr, '*'*25)
    train_copy.loc[:, 'mark'] = 0
    toxic_words = set(list(quantity[quantity>itr].index) + list(words_used_ONLY_in_toxic))
    print(len(toxic_words))
    j=0
    for i in train_copy.index:
        if j%10000==0:
            print(j)
        j+=1
        for word in set(train_copy.loc[i, 'lemm_set'].split('#')):
            if word in toxic_words:
                train_copy.loc[i, 'mark'] = 1
                continue
    a = train_copy[train_copy['mark']==1].shape[0]
    b = f1_score(train_copy[train_copy['mark']==1]['toxic'], train_copy[train_copy['mark']==1]['mark'])
    print(a, b , a*b)

************************* 8 *************************
7125
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
8987 0.9707970682546955 8724.553252404949


In [94]:
middle = quantity[(quantity<=8) & (quantity>0.01)].sort_values(ascending=False)

In [165]:
train_copy_middle = train_copy.copy()
train_copy_middle.loc[:, 'mark_2'] = 0
param = 25
for itr in range(5):
    filling = set(middle[(middle.shape[0]//param)*itr:(middle.shape[0]//param)*(itr+1)].index)
    j=0
    for i in train_copy_middle[train_copy_middle['mark_2']==0].index:
        if j%10000==0:
            print(j)
        j+=1
        for word in set(train_copy_middle.loc[i, 'lemm_set'].split('#')):
            if word in filling:
                train_copy_middle.loc[i, 'mark_2'] = 1
    
    print('*'*25, itr, '*'*25)
    print(train_copy_middle[train_copy_middle['mark_2']==1].shape[0])
    print(train_copy_middle[(train_copy_middle['mark_2']==1) & (train_copy_middle['toxic']==1)].shape[0]/train_copy_middle[(train_copy_middle['mark_2']==1) & (train_copy_middle['toxic']==0)].shape[0])
    print(f1_score(
        train_copy_middle[train_copy_middle['mark_2']==0]['toxic'],
        train_copy_middle[train_copy_middle['mark_2']==0]['mark']
    ))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
************************* 0 *************************
13684
1.3455605073705863
0.778697695233833
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
************************* 1 *************************
14850
1.2141046667660653
0.7781602373887241
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
************************* 2 *************************
15974
1.112404125892621
0.7774153074027603
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
************************* 3 *************************
21408
0.8046025457304223
0.7981082844096543
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
************************* 4 *************************
27002
0.6173704702006588
0.8167453314180175


# 2. Обучение

In [166]:
ntb_checked = set(middle[:(middle.shape[0]//param)*(itr+1)].index)

In [167]:
data_for_model = train_copy_middle[train_copy_middle['mark_2']==1]
data_for_model

,text,toxic,lemmatized,lemm_set,mark,mark_2
91597,Austin Bourke was a meteorologist. He was invo...,0,austin bourke was a meteorologist he was invo...,weird#visitation#agricultural#a#great#thesis#a...,0,1
67544,Don't hide the incriminating photographs you N...,1,don t hide the incriminating photographs you n...,trying#incriminating#think#to#graphic#its#phot...,1,1
131756,"Friendly Discussion ==\n\nDear Kumarrao,\n\nI ...",0,friendly discussion dear kumarrao i had ...,historian#always#of#distinguish#can#p#urs#much...,0,1
127285,I dont like you\nYou are like a Wiki Nazi alwa...,1,i dont like you you are like a wiki nazi alway...,put#always#do#a#some#good#all#delete#people#co...,1,1
64700,"""\n\n Friday FAC \n\nThanks for the comments, ...",0,friday fac thanks for the comments that...,always#weasel#of#disdain#could#values#substitu...,0,1
...,...,...,...,...,...,...
133588,Dangers of Aluminum Bats \n\nWithout citations...,0,dangers of aluminum bats without citations t...,aren#own#of#bat#caused#appear#happened#have#th...,0,1
38205,You stupid and more liar than me you don't wan...,1,you stupid and more liar than me you don t wan...,cease#of#official#renounce#to#took#lie#website...,0,1
45122,"Nah, the leftist bias in this article makes th...",0,nah the leftist bias in this article makes th...,article#wing#a#any#leftist#thing#sad#makes#s#w...,0,1
83278,"""\n\nI was blocked for harrassment and disrupt...",0,i was blocked for harrassment and disruptio...,removes#indicating#wished#block#agreed#quite#o...,0,1


In [168]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = data_for_model['lemmatized'].values.astype('U')


In [169]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
features_train = count_tf_idf.fit_transform(corpus)
target_train = data_for_model['toxic']

In [178]:
model2 = LogisticRegression()
model2.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [174]:
import lightgbm as lgb
model = lgb.LGBMClassifier()
model.fit(features_train, target_train)
model2 = LogisticRegression()
model2.fit(features_train, target_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

# 3. Выводы

In [184]:
test_copy = test.copy()
test_copy.loc[:, 'mark'] = 0
test_copy.loc[:, 'mark_2'] = 0
for i in test_copy.index:
    for word in set(test_copy.loc[i, 'lemmatized'].split()):
        if word in toxic_words:
            test_copy.loc[i, 'mark'] = 1
            continue
        if word in ntb_checked:
        
            test_copy.loc[i, 'mark_2'] = 1
            features_test = count_tf_idf.transform(pd.Series(test_copy.loc[i, 'lemmatized']).astype('U'))
            test_copy.loc[i, 'mark'] = round((model.predict(features_test)[0] + model2.predict(features_test)[0]+0.1)/2)
            continue
test_copy


,text,toxic,lemmatized,mark,mark_2
68588,"who cares, they are worthless scum",1,who cares they are worthless scum,1,1
49379,"@JHJ, the only sense in which CBS Records is S...",0,jhj the only sense in which cbs records is s...,0,0
39969,"And just to be clear, an interview with an ano...",0,and just to be clear an interview with an ano...,0,0
35926,RE: your head page...!\nLOL!,0,re your head page lol,0,0
47779,Nephilim deleted edit \n\nYou state reliable s...,0,nephilim deleted edit you state reliable sou...,0,1
...,...,...,...,...,...
146621,I wrote a letter complaining about you Acroter...,0,i wrote a letter complaining about you acroter...,0,0
145397,Could a section be added to the milan bit \n\n...,0,could a section be added to the milan bit ju...,0,0
126732,"""\n\nI like to know where I stand when someone...",0,i like to know where i stand when someone m...,1,1
29638,"Spaceman spiff,may god be remove your mental p...",0,spaceman spiff may god be remove your mental p...,0,0


In [186]:
true_positive, true_negative, false_positive, false_negative = 0, 0, 0, 0
t = test_copy.copy().reset_index(drop=True)
for i in range(t.shape[0]):
    if (t.loc[i, 'toxic']==1) and (t.loc[i, 'mark']==1):
        true_positive+=1
    if (t.loc[i, 'toxic']==1) and (t.loc[i, 'mark']==0):
        false_negative+=1
    if (t.loc[i, 'toxic']==0) and (t.loc[i, 'mark']==1):
        false_positive+=1
    if (t.loc[i, 'toxic']==0) and (t.loc[i, 'mark']==0):
        true_negative+=1
print(true_positive, true_negative, false_positive, false_negative)

2474 27831 844 766


In [187]:
#print(f1_score(test_copy[test_copy['ntb']==1]['toxic'], test_copy[test_copy['ntb']==1]['mark']))
print(f1_score(test_copy[test_copy['mark_2']==1]['toxic'], test_copy[test_copy['mark_2']==1]['mark']))
print(f1_score(test_copy['toxic'], test_copy['mark']), '-- итог')

0.8090493980904938
0.7544983226593475 -- итог


Из спортивного интереса решил выполнить проект на сервере, а не на локальной машине или не в гугл колабе. Но датасет такой большой, что не то что при попытке запустить берта, при попытке перегнать датасет через tf-idf кернел умирал. Почувствовав себя советским программистом, решил написать, как я думаю, велосипед. Идея заключается в следующем: разделить датасет на 3 части, одну из которых я буду прогонять через модель (использовал tf-idf, lightGBMClassifier, LogisitcsRegression). В двух оставшихся prediction будет состоять из единичек или из нулей.

Как я распределял датасет на части: 
Вообще сначала хотел использовать берта, потому что вот эта строка "использовать BERT необязательно" изрядно меня раззадорила. Но потом я понял, что берт для данного проекта слишком продвинутый инструмент (а еще мне было лениво придумывать как распаковать файл tar.gz): нам не нужно определять смысл каждой фразы, ведь если в высказывании условно содержится одно из всемирно известных четырехбуквенных слов, то она будет токсичной, независимо от посыла. Да, жестко, но мне кажется, справедливо, ведь это не соцсеть, а интернет магазин. Я составил словарь слов которые использовались ТОЛЬКО в токсичных высказываниях (разумеется трейнового датасета), добавил к ним слова которые чаще (более чем в 1 раз) использовались в токсичных высказываниях, чем в нетоксичных. И тут я получил нож в спину от разметчиков. Почему то этот интернет магазин совсем не против обсуждения политики в своих комментариях. Я обнаружил несколько (уверен их было куда больше) политических высказываний, на острые темы, которые можно считать токсичными или нетоксичными в зависимости от позиции визави комментатора. Еще раз повторюсь, это не соцсеть. Почему вот тут такое происходит?

В реальной жизни я бы отправился уточнять задачу, но в данной ситуации датасет не поменять
Поэтому я избрал стратегию которую описал выше. Разделил на три части датасет. "Верхняя" часть -- токсичная, должна была удовлетворять одному условию: максимально возможное значение d1_score (97 меня устроило). "Нтжняя" -- нетоксичная: должна быть максимально большой и опять таки, f1 должна быть значительно выше 75 процентов. Зачем f1 больше 75%? Сейчас  работаю с трейном, я могу добиться точности и близкой к ста на нем. Но когда я перейду на тест, результат может заметно ухудшиться из-за того, что при составлении словаря "токсичных" слов я не пользовался бутстрапом, и словарь для теста может довольно сильно отличаться от трейнового словаря.

"Средняя" часть трейна должна быть небольшой, чтобы кернел не умер, достаточно большой, чтобы с запасом закрыть вот эту "неоднозначную" часть теста и должна содержать примерно поровну 1 и 0 на трейне (а значит что то похожее и на тесте), так регрессия сработает лучше. Постарался максимально соблюсти все эти условия. 

Когда все это выполнил, результат был около 74,3, усреднением двух моделей добился пересечения порога в 75.
Одним из преимуществ моего подхода является возможность внедрения самообучения, поскольку в моей программе есть словарь, в который можно добавлять отсутствующие в нем слова и исправлять соотношение использования в токсичных/нетоксичных случаях. Голь на выдумку хитра конечно, столько оскорблений я до выполнения этого проекта не видел, но в принципе, если внедрить самообучение и длительное время обучать программу, пополняя словарь, можно добиться  почти идеальной ее работы.